# Data Wrangling

## Imports

In [1]:
import pyspark
import pandas as pd
from pyspark.sql.functions import *

spark = pyspark.sql.SparkSession.builder.getOrCreate()

This exercises uses the <b>case.csv, dept.csv, and source.csv</b> files from the San Antonio 311 call dataset.

#### 1. Read the case, department, and source data into their own spark dataframes.

#### 2. Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
- Inspect your folder structure. What do you notice?

#### 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.


In [2]:
df = case = spark.read.csv('case.csv', header=True, inferSchema=True)
print('nrows:', df.count())
stray_animal_cases = df.filter(df.service_request_type == 'Stray Animal').count()
print('stray animal cases:', stray_animal_cases)

nrows: 841704
stray animal cases: 26760


In [3]:
# Rename column
df = df.withColumnRenamed('SLA_due_date', 'case_due_date')

# Convert to better data types
df = (
    df.withColumn('case_late', col('case_late') == 'YES')
    .withColumn('case_closed', col('case_closed') == 'YES')
)
df = df.withColumn('council_district', format_string('%04d', col('council_district')))
df = (
    df.withColumn('case_opened_date', to_timestamp(col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('case_due_date', to_timestamp(col('case_due_date'), 'M/d/yy H:mm'))
)

# Cleanup text data
df = df.withColumn('request_address', lower(trim(col('request_address'))))
# Extract zipcode
df = df.withColumn('zipcode', regexp_extract(col('request_address'), r'\d+$', 0))

# Create a `case_lifetime` feature
df = (
    df.withColumn('case_age', datediff(current_timestamp(), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .drop('case_age', 'days_to_closed')
)

# Join departments and sources
depts = spark.read.csv('dept.csv', header=True, inferSchema=True)
sources = spark.read.csv('source.csv', header=True, inferSchema=True)

df = df.join(depts, 'dept_division', 'left').join(sources, 'source_id', 'left')

# # Train Test Split
# train, test = df.randomSplit([.8, .2], seed=123)
# train, validate, test = df.randomSplit([.7, .15, .15], seed=123)

df.count()

855269

In [4]:
df.createOrReplaceTempView('df')

## Questions

#### 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?



In [5]:
spark.sql('''
SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
FROM df
WHERE NOT case_closed
ORDER BY days_past_due DESC
LIMIT 15
''').show()

+-------------+
|days_past_due|
+-------------+
|         1415|
|         1415|
|         1415|
|         1414|
|         1412|
|         1408|
|         1408|
|         1407|
|         1406|
|         1406|
|         1402|
|         1401|
|         1401|
|         1401|
|         1398|
+-------------+



#### 2. How many Stray Animal cases are there?

In [6]:
df.filter(df.service_request_type == 'Stray Animal').count()

27361

In [7]:
(
    df.groupBy('service_request_type')
    .count()
    .filter(expr('service_request_type == "Stray Animal"'))
    .show()
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|27361|
+--------------------+-----+



#### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [8]:
(
    df.filter(df.dept_division == 'Field Operations')
    .filter(df.service_request_type != 'Officer Standby')
    .count()
)

116295

#### 4. Convert the council_district column to a string column.

In [9]:
# Already completed in data prep

#### 5. Extract the year from the case_closed_date column.

In [10]:
df.select('case_closed_date', year('case_closed_date')).show(5)

+-------------------+----------------------+
|   case_closed_date|year(case_closed_date)|
+-------------------+----------------------+
|2018-01-01 12:29:00|                  2018|
|2018-01-03 08:11:00|                  2018|
|2018-01-02 07:57:00|                  2018|
|2018-01-02 08:13:00|                  2018|
|2018-01-01 13:29:00|                  2018|
+-------------------+----------------------+
only showing top 5 rows



#### 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [11]:
(
    df.withColumn('num_hours_late', df.num_days_late * 24)
    .select('num_days_late', 'num_hours_late')
    .show()
)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
|       -29.74398148| -713.8555555199999|
|       -14.70673611|      -352.96166664|
|       -14.70662037|      -352.95888888|
|       -14.70662037|      -352.95888888|
|       -14.70649306|      -352.95583344|
|       -14.70649306|      -352.95583344|
|       -14.70636574|      -352.95277776|
|          -14.70625|-352.95000000000005|
|       -14.70636574|      -352.95277776|
|       -14.70623843|-352.94972232000003|
|-14.705891199999998|-352.94138879999997|
|       -14.70600694|      -352.94416656|
|       -14.70576389|      -352.93833336|
|       -14.70576389|      -352.93833336|
|       -14.70564815|       -352.9355556|
+-------------------+-------------

#### 7. Join the case data with the source and department data.

In [12]:
sources.show()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows



#### 8. Are there any cases that do not have a request source?

In [13]:
(
    df.select(df.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



#### 9. What are the top 10 service request types in terms of number of requests?

In [14]:
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



#### 10. What are the top 10 service request types in terms of average days late?

In [15]:
(
    df.where('case_late') # just the rows where case_late == true
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509804|51     |
|Donation Container Enforcement        |171.09115313942618|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



#### 11. Does number of days late depend on department?

In [16]:
(
    df.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

+-------------------------+---------+------------+
|dept_name                |days_late|n_cases_late|
+-------------------------+---------+------------+
|Metro Health             |6.5      |854         |
|Solid Waste Management   |7.1      |33729       |
|Trans & Cap Improvements |10.7     |5529        |
|Parks and Recreation     |22.4     |3810        |
|Animal Care Services     |23.4     |23751       |
|Code Enforcement Services|48.1     |25467       |
|Development Services     |67.2     |840         |
|Customer Service         |88.2     |2035        |
|null                     |210.9    |132         |
+-------------------------+---------+------------+



#### 12. How do number of days late depend on department and request type?

In [17]:
(
    df.filter("case_closed")
#     .filter("case_late")
    .groupby("standardized_dept_name", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .sort(asc("days_late"))
    .show(40, truncate=False)
)

+------------------------+--------------------------------------------+---------+-------+
|standardized_dept_name  |service_request_type                        |days_late|n_cases|
+------------------------+--------------------------------------------+---------+-------+
|City Council            |Request for Research/Information            |null     |5      |
|Trans & Cap Improvements|Engineering Design                          |-1413.8  |1      |
|Trans & Cap Improvements|Signal Timing Modification By Engineer      |-1352.0  |22     |
|Animal Care Services    |Stray Animal                                |-998.8   |27346  |
|Parks & Recreation      |Major Park Improvement Install              |-278.3   |271    |
|Trans & Cap Improvements|Sidewalk Cost Sharing Program               |-177.8   |131    |
|DSD/Code Enforcement    |Multi Tenant Exterior                       |-135.7   |84     |
|DSD/Code Enforcement    |CPS Energy Towers                           |-129.8   |511    |
|DSD/Code 